In [1]:
# pad your sequences

from torch.nn.utils.rnn import pad_sequence
from pytorch_lightning.callbacks import LearningRateMonitor
import torch
import numpy as np
import os
import json
import joblib
from torch.utils.data import Dataset,DataLoader,random_split
from itertools import repeat
import pandas as pd
import math
import torch.nn as nn
import torch.nn.functional as F
from torch import linalg as LA
from argparse import Namespace
from numpy import genfromtxt
import os
from torch.optim.lr_scheduler import ReduceLROnPlateau

import pytorch_lightning as pl
import wandb
import logging
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping





In [2]:
#4096 works
import wandb
import logging
from pytorch_lightning.loggers import CSVLogger, TensorBoardLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import os
import shutil

logger = logging.getLogger(__name__)
wandb_logger = lambda dir, version: WandbLogger(
    name="wandb", save_dir=dir, version=version
)
csvlogger = lambda dir, version: CSVLogger(dir, name="csvlogs", version=version)
tblogger = lambda dir, version: TensorBoardLogger(dir, name="tblogs", version=version)

def get_loggers(dir,version,lis=["csv"]):
    lgrs = []
    if "wandb" in lis:
        lgrs.append(wandb_logger(dir, version))
    if "csv" in lis:
        lgrs.append(csvlogger(dir, version))
    if "tb" in lis:
        lgrs.append(tblogger(dir, version))
    return lgrs





In [3]:
# #global variables
# data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'
# annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
# base_annotns_file = data_dir+'annotations/youcookii_annotations_trainval.json'


In [4]:
# with open(base_annotns_file) as json_file:
#     base_annotns = json.load(json_file)

# Data loading

In [5]:

#helper functions
def get_vids(base_dir,split):
    trn_split = base_dir+split
    trn_idlst = []
    trn_vidlst = []

    f = open(trn_split,'r')
    for line in f:
        id_,vid = line.split('/')
        vid = vid.strip('\n')
        trn_idlst.append(id_)
        trn_vidlst.append(vid)
        #print(vid)
        #break
    f.close()
    return trn_idlst,trn_vidlst
   
    
#gives video feats
def get_features(data_dir,split='val',feat_dir='/common/users/vk405/feat_csv/'):
    #feat_dir = data_dir
    splits_dir = data_dir+'splits/'
    if split == 'val':
        feat_split_dir = feat_dir+'val_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'val_list.txt')  
    elif split == 'train':
        feat_split_dir = feat_dir+'train_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'train_list.txt') 
    elif split == 'test':
        feat_split_dir = feat_dir+'test_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'test_list.txt')
    else:
        raise NotImplementedError(f'unknown split: {split}')     
    feat_list = {}
    vid_dtls = []
    for num,name in zip(vid_num,vid_name):
        feat_loc = os.path.join(feat_split_dir, f'{num}/{name}/0001/')
        #import pdb;pdb.set_trace()
        if os.path.isdir(feat_loc):
            feat_files = feat_loc + os.listdir(feat_loc)[0]
            feat_list[name] = feat_files
            #feat_list.append(feat_files)
            vid_dtls.append((num,name))
        else:
            print(f"video : {num}/{name} not found")
    assert len(feat_list) == len(vid_dtls),"get-features is giving incorrect features"
    return feat_list,vid_dtls


def get_labels(ids,annotns_file):
    #gives annotations corresponding to the ids(videos) given
    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        #print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns:
                #import pdb;pdb.set_trace()
                duration = annotns[vidname]['duration']
                annot = annotns[vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    st_end = [interval[0],interval[-1]]
                    sent = segment_info['sentence']
                    labels.append((st_end,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

Raw time annotations are converted to corresponding frame-level features.

In [6]:

# feat_locs,vids = get_features(data_dir,split='train')

In [7]:
# labels = get_labels(vids,annotns_file)

In [8]:
# labels['GLd3aX16zBg']

In [9]:
# base_annotns['database']['GLd3aX16zBg']['annotations']

In [10]:
# vid_file= feat_locs['GLd3aX16zBg']
# vid = genfromtxt(vid_file, delimiter=',')

Use contrastive/triplet loss. Train the model to reduce this loss over every negative sample of video frames. Keep It simple and sample a random video frame(non-alighning).

In [11]:


#dataset
# Dataset/loader
# This is newer version
class YoucookDset2(Dataset):
    global_txt = joblib.load(os.path.join('/common/home/vk405/Projects/Crossmdl/Data/YouCookII/','emb.joblib'))
    global_imgfeats = joblib.load('/common/users/vk405/feat_csv/global_imgfeats.joblib')
    global_map = joblib.load('/common/users/vk405/feat_csv/global_map.joblib')
    def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
        ,split='train',framecnt=499):
        self.feat_locs = {}
        self.split = split
        self.data_dir = data_dir
        self.framecnt = framecnt
        #self.use_precomp_emb = use_precomp_emb
        self.text_emb = None
        if self.split != 'test':
            self.annotns_file = data_dir+'annotations/segment_youcookii_annotations_trainval.json'
        else:
            raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
        # if self.use_precomp_emb:
        #     self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))
        #feat_locs = {'Ysh60eirChU': location of the video}
        #import pdb;pdb.set_trace()
        self.feat_locs,vids = get_features(self.data_dir,split=self.split)
        #import pdb;pdb.set_trace()
        assert len(vids) == len(self.feat_locs),"features are wrong"
        #import pdb;pdb.set_trace()
        #label_info = get_labels(vids,self.annotns_file)
        #self.labelencoder = LabelEncoder2()
        self.final_labels = get_labels(vids,self.annotns_file)
        #self.labelencoder.fit_transform(label_info)
        
        #regress_labels(label_info)
        #(vid_id,seg_id)
        self.data,self.max_seq_ln= self.update_data()

                
            
    def __len__(self):
        return len(self.data)

    def overlap_frac(self,base_rng,tst_rng):
        #1.Returns the fraction of frames that are overlapping in tst_rng with base_rng
        #2.both ends inclusive
        sz = tst_rng[-1]-tst_rng[0]+1
        lbl_ids = set(np.arange(base_rng[0],base_rng[-1]+1))
        frame_ids = set(np.arange(tst_rng[0],tst_rng[-1]+1))
        inter = frame_ids.intersection(lbl_ids)
        assert sz != 0,"base frame rng is zero"
        return len(inter)/sz




    def update_data(self):
        data = []
        max_cnt = 50
        mx_seq_ln = 0
        for key in self.final_labels:
            segments = self.final_labels[key]
            for ind,seg in enumerate(segments):
                #trn_points = []
                st_end,txt,vid_len = seg
                main_seg = (key,self.feat_locs[key],st_end[0],st_end[-1],ind,1.0)
                # to keep track of max sequence length in the data
                sq_len = st_end[-1]-st_end[0]+1
                mx_seq_ln = max(mx_seq_ln,sq_len)

                data.append(main_seg)
                frame_width = st_end[-1]-st_end[0] + 1
                extra_frames = []
                #st_end[-1]+1
                for cnt,new_st in enumerate(range(st_end[0]+1,499)):
                    #forward sliding
                    new_end = new_st+frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                #st_end[0]
                for cnt,new_end in enumerate(range(st_end[-1],0,-1)):
                    #backward sliding
                    new_st = new_end-frame_width
                    if (cnt<max_cnt)and (0<=new_st<self.framecnt and 0<=new_st<self.framecnt):
                        extra_frames.append((new_st,new_end))
                
                #import pdb;pdb.set_trace()
                zero_overlap_frames = []
                for ex_seg in extra_frames:
                    label = self.overlap_frac(st_end,ex_seg)
                    # only negative samples
                    if label == 0.00:
                        zero_overlap_frames.append(ex_seg)
                        #data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,label))
                        #break
                if len(zero_overlap_frames):
                    frame_id = np.random.randint(len(zero_overlap_frames))
                    ex_seg = zero_overlap_frames[frame_id]
                    #print(f"main_seg:{main_seg},ex_seg:{ex_seg}")
                    data.append((key,self.feat_locs[key],ex_seg[0],ex_seg[-1],ind,0.0))
                    sq_len = ex_seg[-1]-ex_seg[0]+1
                    mx_seq_ln = max(mx_seq_ln,sq_len)
                    
        return data,mx_seq_ln
    
    def read_data_global(self,data):
        vid_id,vid_loc,start,end,segid,label = data
        ind = self.global_map[vid_id]
        img_feat = self.global_imgfeats[ind][start:end+1]
        txt_feat = self.global_txt[vid_id][segid]
        pad_img = np.concatenate([img_feat,np.zeros((self.max_seq_ln-img_feat.shape[0],img_feat.shape[-1]))])
        return pad_img,txt_feat,len(img_feat),label

    def __getitem__(self,idx):
        return self.read_data_global(self.data[idx])
        

           



        

In [12]:
# ydata = YoucookDset2()


In [13]:


# train_loader = DataLoader(ydata,\
#             batch_size=170,shuffle=True)

In [14]:
# batch = next(iter(train_loader))

In [15]:

# img,txt,lengths,labels = batch

In [16]:
# img.shape,lengths

In [17]:
# bilstm = nn.LSTM(512, 100, 2, bidirectional=True,batch_first=True)

In [18]:
# packed = nn.utils.rnn.pack_padded_sequence(img.float(), \
#     lengths, batch_first=True, enforce_sorted=False)


In [19]:
# hiddens, (final_h, final_c) = bilstm(packed)

In [20]:
# hiddens, lengths = nn.utils.rnn.pad_packed_sequence(hiddens, batch_first=True)  # (batch_size, max_length, 2*dim_lstm) 

In [21]:
class VideoLstmEncoder(nn.Module):
    # LSTM ENCODER OVER VIDEO FRAMES
    def __init__(self,vid_frm_sz=512,vid_hid_sz=512,\
        lyrs=2,bidirectional=True,drop=0.5,fin_sz=256):
        super().__init__()
        self.vid_frm_sz=vid_frm_sz
        self.vid_hid_sz=vid_hid_sz
        self.lyrs=lyrs
        self.bidirectional=bidirectional
        self.fin_sz=fin_sz
        ####lstm##
        #import pdb;pdb.set_trace()
        self.lstm = nn.LSTM(self.vid_frm_sz,self.vid_hid_sz,\
            self.lyrs,bidirectional=self.bidirectional,batch_first=True)
            
        self.drop = nn.Dropout(drop)
        # This gives a fixed size output and adds a non-linearity.
        if self.bidirectional:
            self.lin = nn.Linear(self.vid_hid_sz*2,self.fin_sz)
        else:
            self.lin = nn.Linear(self.vid_hid_sz,self.fin_sz)
        self.act  = nn.ReLU()
        
    def forward(self,frame_chunk,lengths):
        packed_frames = nn.utils.rnn.pack_padded_sequence(frame_chunk.float(), \
               lengths, batch_first=True, enforce_sorted=False)
        hiddens, (final_h, final_c) = self.lstm(packed_frames)
        #import pdb;pdb.set_trace()
        hiddens, lengths = nn.utils.rnn.pad_packed_sequence(hiddens, batch_first=True) 
        vid_emb = hiddens.sum(1)/lengths.unsqueeze(1)  
        
        vid_emb = self.act(self.lin(self.drop(vid_emb)))
        return vid_emb
        


        

In [22]:
# #testing

# lstmencoder = VideoLstmEncoder()
# out = lstmencoder(img,lengths)
# out.shape

In [23]:
class VideoTextEncoder(pl.LightningModule):
    def __init__(self,hparams):
        super().__init__()
        self.save_hyperparameters(hparams)

        self.videoencoder = VideoLstmEncoder(**hparams.video_cfg)
        self.textencoder = nn.Sequential(*self.linear_encoder(**hparams.text_cfg))
        
        self.shared = nn.Linear(hparams.shared_inptsz,hparams.shared_outsz)

    def forward(self,vid_frames,lengths,txts):
        txt_emb = self.textencoder(txts.float())
        vid_emb = self.videoencoder(vid_frames,lengths)
        loss_cfg = self.hparams.loss_cfg

        if loss_cfg['loss_type'] == 'classification':
            if loss_cfg['mode'] == 'concat':
                #late fusion of two modalities to give class-probability
                #import pdb;pdb.set_trace()
                fin_emb = self.shared(torch.concat([txt_emb,vid_emb],axis=-1))
                return F.sigmoid(fin_emb)
                
        elif loss_cfg['loss_type'] == 'triplet':
            txt_emb = self.shared(txt_emb)
            vid_emb = self.shared(vid_emb)
            return vid_emb,txt_emb

    def training_step(self,batch,batch_idx):
        loss_cfg = self.hparams.loss_cfg
        vid_frames,txts,lengths,labels = batch
        if loss_cfg['loss_type'] == 'classification':
            sim_score = self(vid_frames,lengths,txts)
            loss = F.binary_cross_entropy(sim_score.squeeze().float(),\
                     labels.squeeze().float())
            
        else:
            raise NotImplementedError
        self.log("train_loss",loss,on_step=True)
        return loss
    
    def validation_step(self,batch,batch_idx):
        # same as training_step but happens with in torch.no_grad()
        loss_cfg = self.hparams.loss_cfg
        vid_frames,txts,lengths,labels = batch
        if loss_cfg['loss_type'] == 'classification':
            sim_score = self(vid_frames,lengths,txts)
            loss = F.binary_cross_entropy(sim_score.squeeze().float(), \
                labels.squeeze().float())
            s = sim_score.squeeze().float()
            l = labels.squeeze().float()
            preds = (torch.where(s > 0.5, 1.0, 0.0) == l)
            val_acc = torch.mean(preds.type(torch.DoubleTensor))


            
        else:
            raise NotImplementedError
        self.log("validation_loss",loss,on_step=False, on_epoch=True)
        self.log("validation_acc",val_acc,on_step=False, on_epoch=True)
        return loss


    def configure_optimizers(self):
        lr = self.hparams.lr 
        optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        lr_scheduler = ReduceLROnPlateau(optimizer, 'min',\
            factor=0.8,patience=5,min_lr=5e-5)
        return {
        "optimizer": optimizer,
        "lr_scheduler": {
            "scheduler": lr_scheduler,
            "monitor": "validation_loss"
               }
              }

        

    

    def triplet_loss(self,img,txt,anch,reduce='mean'):
        #𝐿𝑐𝑜𝑠(𝒂,𝒑,𝒏)=max[𝑑(𝒂,𝒏)−𝑑(𝒂,𝒑)+𝜖,0]
        loss_cfg = self.hparams.loss_cfg
        eps = loss_cfg['eps'] if 'eps' in loss_cfg else 0.3
        
        im_norm,txt_norm,anch_norm = LA.norm(img,dim=-1).reshape(img.shape[0],1),\
        LA.norm(txt,dim=-1).reshape(txt.shape[0],1),\
            LA.norm(anch,dim=-1).reshape(anch.shape[0],1)
        normd_img = img/im_norm
        normd_txt = txt/txt_norm
        normd_anch = anch/anch_norm

        cos_sim_p = torch.sum(normd_img*normd_txt,dim=-1)
        cos_sim_n = torch.sum(normd_anch*normd_txt,dim=-1)
        if 'use_mse' in loss_cfg and loss_cfg['use_mse']:
            #print(f"USING MSE LOSS")
            loss  = F.mse_loss(img,txt)
        else:
            unclipped_loss = cos_sim_n-cos_sim_p+eps
            loss = torch.relu(unclipped_loss)
        if reduce == 'mean':
            return torch.mean(loss),(torch.mean(cos_sim_n),torch.mean(cos_sim_p))

    
    def linear_encoder(self,init_sz,fin_sz,lyrs,activation='relu'):
        # Gives #lyrs of liner+activation layers with hidden size as a linear interpolation
        #between init_sz and fin_sz
        lyr_lst = []
        hd_sz = [round(ele) for ele in np.linspace(init_sz,fin_sz,lyrs+1)][1:]
        lst_sz = init_sz
        for i in range(lyrs):
            lyr_lst.append(nn.Linear(lst_sz,hd_sz[i]))
            if activation == 'relu':
                lyr_lst.append(nn.ReLU())
            lst_sz = hd_sz[i]
        return lyr_lst


            
    

In [24]:
(0.8**10)*(0.001)

0.00010737418240000006

In [25]:
#cfg
cfg = Namespace(
    seed = 0,
    version = 'retrieval',
    artifacts_loc = "/common/home/vk405/Projects/Crossmdl/nbs/Recipe/",
    data_dir = "/common/home/vk405/Projects/Crossmdl/Data/Recipe/",
    mode = 'train',
    video_cfg = {'vid_frm_sz':512,'vid_hid_sz':128,\
        'lyrs':1,'bidirectional':True,'drop':0.5,'fin_sz':256},
    text_cfg = {'init_sz':768,\
        'fin_sz':256,'lyrs':2,'activation':'relu'},
    loss_cfg = {'loss_type':'classification','mode':'concat'},
    shared_inptsz = 512,
    shared_outsz = 1,
    lr = 0.0004,
    loggers = ["csv"],
    cbs = ["checkpoint","early_stop"],
    trainer = {'log_every_n_steps': 50,
    'max_epochs': 100},
    checkpoint = {"every_n_epochs": 1,
    "monitor": "val_recall_1"},
    early_stop = {"monitor":"val_recall_1","patience":2,"mode":'max'},
    batch_size=512,
    val_batch_size = 2000


)

In [26]:
# #testing

# ydata = YoucookDset2()

# train_loader = DataLoader(ydata,\
#             batch_size=10,shuffle=True)

# batch = next(iter(train_loader))
# img,txt,lengths,labels = batch

# net = VideoTextEncoder(cfg)
# net

In [27]:
# #testing
# net.textencoder(txt.float()).shape,net.videoencoder(img,lengths).shape

In [28]:
# out = net(img,lengths,txt)

In [29]:
# #testing
# out,labels

In [30]:
# len(ydata)

In [31]:
# #testing
# ydata = YoucookDset2()


# ydata_ss = torch.utils.data.Subset(ydata,range(2000))


# train_loader = DataLoader(ydata_ss,\
#             batch_size=170,shuffle=True)

In [32]:
# #testing
# #track_grad_norm=2,gradient_clip_val=0.5


# net = VideoTextEncoder(cfg)
# net

In [33]:
# trainer = pl.Trainer(deterministic=True,**cfg.trainer
#         )
# trainer.fit(net, train_loader)

In [34]:

# batch_d = []
# cnt = 0
# for ele in train_loader:
#     batch_d.append(ele)
#     cnt += 1
#     if cnt >= 5:
#         break



In [35]:
# img,txt,lengths,labels = batch_d[-1]


In [36]:
#trainer.model(img,lengths,txt)

In [37]:
def run(cfg):
    pl.seed_everything(cfg.seed)
    dir = cfg.artifacts_loc
    version = str(cfg.version)
    logger_list = get_loggers(dir, version,cfg.loggers)
    cbs = []
    if "early_stop" in cfg.cbs:
        #? does'nt really work atm
        params = cfg.model.cbs.early_stop
        earlystopcb = EarlyStopping(**params, min_delta=0.00, verbose=False)
        cbs.append(earlystopcb)
    if "checkpoint" in cfg.cbs:
        store_path = dir + "ckpts/" + str(cfg.version) + "/"
        isExist = os.path.exists(store_path)
        if isExist and os.path.isdir(store_path) and ('retrain' not in cfg and not cfg.retrain):
            shutil.rmtree(store_path)
        # then create fresh
        if not isExist:
            os.makedirs(store_path)
        fname = "{epoch}-{train_loss:.2f}"
        params = cfg.checkpoint
        checkptcb = ModelCheckpoint(**params, dirpath=store_path, filename=fname)
        cbs.append(checkptcb)
    if "wandb" in cfg.loggers:
        wandb.init(project="videoretrieval", config=cfg)
    if cfg.mode == 'train':
        split = cfg.data_split if 'data_split' in cfg else cfg.mode
        youcookdata = YoucookDset2(data_dir=cfg.data_dir,split=split)
        vld_sz = int(cfg.val_split*len(youcookdata))
        trn_sz = len(youcookdata)-vld_sz
        trn_data,vld_data = random_split(youcookdata,[trn_sz,vld_sz])
        
        if cfg.use_precomp_emb:
            pass
            #global_txt = joblib.load(os.path.join(data_dir,'emb.joblib'))
            #collate_wrapper = MyCollator(txtemb=global_txt)
        #pin_memory=True
        train_loader = DataLoader(trn_data,\
            batch_size=cfg.batch_size,shuffle=True)

        valid_loader = DataLoader(vld_data,batch_size=cfg.batch_size)
           
        net = VideoTextEncoder(cfg)
        #gpus=1,gpus=3,accelerator='ddp'
        if 'retrain' in cfg and cfg.retrain:
            path = cfg.artifacts_loc+'ckpts/'+cfg.version +'/'
            weight = os.listdir(path)[-1]
            ck_path = path+weight
            print(f"loading from:{ck_path}")
            net = net.load_from_checkpoint(ck_path)

        lr_monitor = LearningRateMonitor(logging_interval='epoch')
        trainer = pl.Trainer(
            logger=logger_list,callbacks=cbs+[lr_monitor],deterministic=True,\
                gradient_clip_val=0.8,**cfg.trainer
        )
        trainer.fit(net, train_loader,valid_loader)
        return trainer
        #trainer.tune(net,train_loader)
            
    else:
        pass
    

In [38]:
#cfg
cfg = Namespace(
    seed = 0,
    version = 'LstmEncoder',
    artifacts_loc = "/common/home/vk405/Projects/Crossmdl/nbs/",
    data_dir = "/common/home/vk405/Projects/Crossmdl/Data/YouCookII/",
    mode = 'train',
    val_split = 0.2,
    video_cfg = {'vid_frm_sz':512,'vid_hid_sz':128,\
        'lyrs':1,'bidirectional':True,'drop':0.5,'fin_sz':256},
    text_cfg = {'init_sz':768,\
        'fin_sz':256,'lyrs':2,'activation':'relu'},
    loss_cfg = {'loss_type':'classification','mode':'concat'},
    shared_inptsz = 512,
    shared_outsz = 1,
    lr = 0.0004,
    loggers = ["csv","wandb"],
    cbs = ["checkpoint"],
    trainer = {'log_every_n_steps': 5,
    'max_epochs': 200},
    checkpoint = {"every_n_epochs": 1,
    "monitor": "validation_loss"},
    early_stop = {"monitor":"validation_loss","patience":2,"mode":'min'},
    batch_size=512,
    val_batch_size = 2000,
    retrain=False,
    use_precomp_emb=True


)

In [39]:
trainer = run(cfg)

Global seed set to 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: vin136 (use `wandb login --relogin` to force relogin)
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/IPython/html.py:12: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  warn("The `IPython.html` package has been deprecated since IPython 4.0. "


GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:1584: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  rank_zero_warn(
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(

  | Name         | Type             | Params
--------------------------------------------------
0 | videoencoder | VideoLstmEncoder | 723 K 
1 | textencoder  | Sequential       | 525 K 
2 | shared       | Linear           | 513   
--------------------------------------------------
1.2 M     Trainable params


Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation sanity check:  50%|█████     | 1/2 [00:02<00:02,  2.37s/it]

/common/home/vk405/.local/lib/python3.8/site-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Global seed set to 0
/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 80 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 157:  13%|█▎        | 5/38 [02:15<14:52, 27.03s/it, loss=0.165, v_num=oder] 